# This notebooks helps to label the position of a Figure on a PDF
We want to know:

- The page where the Figure can be found
- The x0,x1,y0,y1 position of the figure in the page

All the information above will be stored on the `annotation.json` file
This file has the following structure:
```
     "<ID>": {
        "pdf_path": "<path_to_pdf_on_dabase>",
        "Watermark": <bool>,
        "<figx>": {
            "p": <page_number>,
            "y0": <position>,
            "y1": <position>,
            "x0": <position>,
            "x1": <position>
        },
        "missing": [< list_of_figure_withou_label>],
        "all_figures": [
            <list_of_all_figure_downloaded>
        ]
    },
```

Import Cell

In [14]:
# import the necessary packages
import os
from glob import glob
import numpy as np
import fitz
import io
from PIL import Image
import io
import pandas as pd
import cv2
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import json
import screeninfo
import pyperclip

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [15]:
# Debug panel
def display_panel(imgs):
    """
    Easy way to create a panel to check any image from imgs list
    
    Parameters
    ----------
    imgs: list
        List of image on opencv format (numpy)
    
    Return
    ------
    panel: PIL object
        A PIL Image object with the imgs inputed in a panel format
    """
    # Total imgs
    total_imgs = len(imgs)

    # Get min shape
    min_shape = sorted( [(np.sum(i.shape[:2]), i.shape[:2] ) for i in imgs])[0][1]
    rows,cols = min_shape
    # Adding border to imgsa
    bordersize = 2

    imgs_aux = [ cv2.copyMakeBorder(
                            cv2.resize(im,(cols,rows)), # Resize img
                            top=bordersize,
                            bottom=bordersize,
                            left=bordersize,
                            right=bordersize,
                            borderType=cv2.BORDER_CONSTANT,
                            value=[69, 87, 96]

                    ) for im in imgs]
    # number of img in vertical
    nv_imgs = int(np.floor(np.sqrt(total_imgs))) 
    # number of img in horizontal
    nh_imgs = int(np.ceil(total_imgs/nv_imgs))

    # Empty imgs
    empty =  nv_imgs*nh_imgs - total_imgs


    # Image Panel on array format
    img_ar = imgs_aux[0]

    # Insert missing imgs to complete a rectangle imgs
    if empty > 0:
        for i in range(empty):
            ii = np.zeros_like(img_ar)
            ii[:] = [69, 87, 96]
            imgs_aux += [ii]

    # Concatenate all imgs
    for i in range(1,len(imgs_aux)):
        img_ar = np.concatenate([img_ar,imgs_aux[i]],axis=0)
    # Reshape properly to (nh_imgs nv_imgs)
    r = img_ar.reshape((nh_imgs,-1)+img_ar.shape[1:])
    panel =  np.hstack(r)

    return  panel

def print_pages(doc, output):
    for n, page in enumerate(doc):
        rendered_page = page.getPixmap(alpha=False,annots=False).getImageData()
        pg_name = f'{output}/{n+1}.png'
        with open(pg_name,'wb') as fp:
            fp.write(rendered_page)

## Checking the figures locations

In [16]:



#Global Var
screen_id = 0
is_color = True

global ix, iy ,drawing, image, image_index,window_name
window_name = ""
ix = -1
iy = -1
drawing = False
image = None
image_index = 0
  
# Draw Callback
def draw_rectangle_with_drag(event, x, y, flags, param): 
      
    global ix, iy, drawing, image, image_index, window_name
   
    if event == cv2.EVENT_LBUTTONDOWN: 
        drawing = True
        ix = x 
        iy = y 
        
    elif event == cv2.EVENT_MOUSEMOVE: 
        if drawing == True: 
            image_moving = image.copy()
            cv2.rectangle(image_moving, pt1 =(ix, iy), 
                          pt2 =(x, y), 
                          color =(255,0, 255), 
                          thickness = 3)
            
            cv2.imshow(window_name, image_moving)
      
    elif event == cv2.EVENT_LBUTTONUP: 
        drawing = False
        cv2.rectangle(image, pt1 =(ix, iy), 
                      pt2 =(x, y), 
                      color =(255,0, 255), 
                      thickness =3) 
        cv2.imshow(window_name, image)
        dic = """"figx": {
            "p": %d,
            "y0": %d,
            "y1": %d,
            "x0": %d,
            "x1": %d
        },""" % (image_index+1,min(iy,y),max(y,iy),min(ix,x),max(x,ix))
        # copy to clipbord
        pyperclip.copy(dic)
        print(dic)
        
        
        
#Functions
def save_df(df):
    df.to_csv("annotation.csv",index = False)
    
def load_files():
    with open('annotation.json', 'r') as fp:
        annotation_json = json.load(fp)
    data = pd.read_csv("annotation.csv",index_col = False)
    return annotation_json, data

def get_pdf_index (pdf_path,data):
    return data.loc[ data["PDFPATH"] == pdf_path ]['ID'].values[0]

def show_image(page_list):
    global image, window_name, image_index
    image = cv2.cvtColor(page_list[image_index], cv2.COLOR_RGB2BGR)

    text = "Page"
    color = (20,20,200)      
    cv2.putText(image, "{} {}/{} ".format(text, image_index+1,len(page_list)), (10, 50),                                                                                                                                             
    cv2.FONT_HERSHEY_TRIPLEX, 0.5, color, 1)                                                                                                                                                                                      
    cv2.imshow(window_name, image)

def page_annotation(pdf_path):
    """
    Open a Windowns of cv2 to display the pdf page. After this, it allows the user
    to label the position of a figure by drawing a rectangle on the screen.
    
    Warning
    -------
    This function make use of global var.
    
    Parameters
    ---------
    pdf_path: <str> 
        Path to the file of the PDF
    """
    
    annotation_json, annotated_data =  load_files()
    pdf_index = get_pdf_index(pdf_path,annotated_data)
    figures_state = annotation_json[str(pdf_index)]
    checked = annotated_data.loc[pdf_index]['CHECKED']
   
    
    screen = screeninfo.get_monitors()[screen_id]
    width, height = screen.width, screen.height
    global window_name
    window_name = f'PDF INDEX - {pdf_index}'
    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.moveWindow(window_name, screen.x - 1, screen.y - 1)
    cv2.setWindowProperty(window_name, int(cv2.WND_PROP_FULLSCREEN/2),int(cv2.WINDOW_FULLSCREEN/2))
    cv2.setMouseCallback(window_name, draw_rectangle_with_drag) 
    global refPt
    global cont 
    cont = 1
    refPt = [None,None]
        
    page_list = generate_imgs_with_bb(pdf_path,figures_state)
    global image_index
    image_index = 0
    show_image(page_list)
    while(1):
        key = cv2.waitKey(20)
                
        # Keymap for forward and backward
        if key==ord('d'):  # normally -1 returned,so don't print it
            image_index +=1
            image_index = image_index % len(page_list)
            
            annotation_json, annotated_data =  load_files()
            figures_state = annotation_json[str(pdf_index)]
            page_list = generate_imgs_with_bb(pdf_path,figures_state)
            show_image(page_list)
            
        elif key==ord('a'):
            image_index -= 1
            image_index = image_index % len(page_list)
            
            annotation_json, annotated_data =  load_files()
            figures_state = annotation_json[str(pdf_index)]
            page_list = generate_imgs_with_bb(pdf_path,figures_state)
            show_image(page_list)
        
        elif key==ord(' '):
            annotation_json, annotated_data =  load_files()
            figures_state = annotation_json[str(pdf_index)]
            page_list = generate_imgs_with_bb(pdf_path,figures_state)
            show_image(page_list)
            
        elif key==27:
            print("ESCAPE")
            break # else print its value
    cv2.destroyAllWindows()
    
def generate_imgs_with_bb(pdfpath,fig_state):
    """
    To visualize the downloaded figures from a pdf
    Make sure to put all the extracted/downloaded figure
    on the path 'pdfpath'/figures/
    """

    # Check number of figures
    figs_path = os.path.dirname(pdfpath)+"/figures/"
    figs = glob(figs_path+"/fig*[!.txt]")
    figs.sort()
    figs = [os.path.basename(f) for f in figs]
    figs = [f[:f.rfind(".")] for f in figs]
    
    # Adding supplementary image into figs
    for sfig in fig_state.keys():
        if 'figS' in sfig:
            figs.append(sfig)
    # Open Document and print the location of the bounding on each page
    page_figures = []
    doc_dummy = fitz.open(pdfpath)
    for page_dummy in doc_dummy:
        # Have to open again to due error
        pn = page_dummy.number + 1    
        doc = fitz.open(pdfpath)
        page = doc[page_dummy.number]
        # Compare the relevant images with the Figures from the official repository
        img_fig = page.getPixmap(alpha=False,annots=False).getImageData()
        img_fig = np.array(Image.open(io.BytesIO(img_fig)))

        for f in figs:
            if f in fig_state.keys():
                fig = fig_state[f]
                if fig['p'] == pn:

                    start_point = (fig['x0'], fig['y0']) 
                    end_point = (fig['x1'], fig['y1'])
                    if 'figS' in f:
                        color = (40,150,40)
                    else:
                        color = (150, 150, 0) 
                    thickness = 5
                    # Using cv2.rectangle() method 
                    # Draw a rectangle with blue line borders of thickness of 2 px 
                    cv2.rectangle(img_fig, start_point, end_point, color, thickness)
                    f = f.replace("fig","")
                    cv2.putText(img_fig ,f, (int((fig['x0'] + fig['x1'])/2), int((fig['y0'] + fig['y1'])/2)),\
                                cv2.FONT_HERSHEY_SIMPLEX, 5, color, 8)
        page_figures.append(img_fig)
        doc.close()
        
    return page_figures

# Run Cell

In [17]:
# Run annotation tool
pdf_path = pyperclip.paste() # Get pdf path from the clipboard
page_annotation(pdf_path)

"figx": {
            "p": 2,
            "y0": 75,
            "y1": 297,
            "x0": 321,
            "x1": 555
        },
ESCAPE


----